In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from pptx import Presentation
from pptx.util import Inches


data = pd.read_csv('customer_booking_preprocessed.csv')
X = data.drop('booking_complete', axis=1)
y = data['booking_complete']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


feature_importances = model.feature_importances_
feat_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)


plt.figure(figsize=(10,6))
sns.barplot(x='Importance', y='Feature', data=feat_importance_df, palette='viridis')
plt.title('Feature Importance in Random Forest Model')
plt.xlabel('Relative Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.close()


cv_scores = cross_val_score(model, X, y, cv=5)
avg_cv_score = np.mean(cv_scores)


prs = Presentation()
slide_layout = prs.slide_layouts[5]  # Title only
slide = prs.slides.add_slide(slide_layout)

# Add title
slide.shapes.title.text = "Customer Booking Prediction - Key Insights"

# Add summary bullets
left = Inches(0.5)
top = Inches(1.5)
width = Inches(8)
height = Inches(2)
txBox = slide.shapes.add_textbox(left, top, width, height)
tf = txBox.text_frame

tf.text = f"• Model achieved ~{avg_cv_score:.2%} average cross-validation accuracy."

p = tf.add_paragraph()
p.text = "• Top predictors: purchase lead time, flight hour, wants extra baggage, length of stay."

p = tf.add_paragraph()
p.text = "• Business can target early planners and upsell extras for higher conversion."

# Add feature importance image
img_left = Inches(1)
img_top = Inches(3)
img_width = Inches(6)
img_height = Inches(3.5)
slide.shapes.add_picture('feature_importance.png', img_left, img_top, img_width, img_height)

# Save presentation
prs.save('Customer_Booking_Model_Summary.pptx')
print("✅ PowerPoint slide saved as 'Customer_Booking_Model_Summary.pptx'")